In [23]:
import pandas as pd
import numpy as np

from html.parser import HTMLParser
from bs4 import BeautifulSoup
import requests

In [35]:
!pip install geocoder

     |████████████████████████████████| 102kB 13.8MB/s 
     |████████████████████████████████| 81kB 18.5MB/s 


Load html page from wikipedia

In [24]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

Parse the table from html to pandas and save it to list

In [25]:
list = pd.read_html(page, header=0, attrs={"class":"wikitable sortable"})[0]

Drop all rows where Borough is Not assigned

In [26]:
list = list[list.Borough != 'Not assigned']

Group rows by Postcode and Borught and Neighbourhood consolidate to string separated by ,.

In [27]:
list = list.groupby(['Postcode', 'Borough'],as_index=False)['Neighbourhood'].agg(', '.join)

Search all rows where Neighbourhood is not assigned and fill them with a value of Borough

In [28]:
for i in range(len(list)):
    if list['Neighbourhood'].values[i] == 'Not assigned':
        list['Neighbourhood'].values[i] = list['Borough'].values[i]

Add Latitude and Longitude columns to the list

In [29]:
list["Latitude"] = ""
list["Longitude"] = ""

Import geocoder library

In [91]:
import geocoder # import geocoder

Define a function for getting coordinates via geocoder.arcgis

In [141]:
def getCoordinates(postcode, borough, neighbourhood):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis("%s, %s, %s" % (postcode, borough, neighbourhood))
        lat_lng_coords = g.latlng
    
    coordinates = {
    "lat" : lat_lng_coords[0],
    "lon" : lat_lng_coords[1]
        }
    return coordinates

For each row on the list fill in latitide nad longitude by calling getCoordinates function

In [142]:
for i in range(len(list)):
    postcode = list['Postcode'].values[i]
    borough = list['Borough'].values[i]
    neighbourhood = list['Neighbourhood'].values[i]
    c = getCoordinates(postcode, borough, neighbourhood)
    list['Latitude'].values[i] = c['lat']
    list['Longitude'].values[i] = c['lon']
        

Create a DataFrame from the list

In [143]:
neighborhoods = pd.DataFrame(list)

Show the result

In [144]:
neighborhoods

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8098,-79.2208
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7857,-79.1587
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.766,-79.1854
3,M1G,Scarborough,Woburn,43.7684,-79.2176
4,M1H,Scarborough,Cedarbrae,43.7573,-79.2245
5,M1J,Scarborough,Scarborough Village,43.7431,-79.2317
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7136,-79.2603
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.726,-79.2833
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7236,-79.235
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.7038,-79.2506


Show size of the DataFrame

In [145]:
neighborhoods.shape

(103, 5)